In [2]:
import sys
sys.executable


'd:\\ZenithQuest\\leadership-coaching-reflection-intelligence\\venv\\Scripts\\python.exe'

In [3]:
import pandas as pd

df = pd.read_csv("sample_input.csv")
df.head()


,participant_id,program_name,week_number,question,response,submission_timestamp
0,P001,First-Time Manager Program,1,What leadership challenge did you face this week?,I struggled to delegate tasks because I was wo...,01-09-2025 10:15
1,P002,First-Time Manager Program,1,What leadership challenge did you face this week?,Managing conflicting priorities was difficult ...,01-09-2025 11:02
2,P003,First-Time Manager Program,1,What leadership challenge did you face this week?,I had trouble giving feedback to a team member...,01-09-2025 11:45
3,P004,First-Time Manager Program,1,What leadership challenge did you face this week?,I was not clear about my role as a manager and...,01-09-2025 12:10


In [4]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   participant_id        4 non-null      str  
 1   program_name          4 non-null      str  
 2   week_number           4 non-null      int64
 3   question              4 non-null      str  
 4   response              4 non-null      str  
 5   submission_timestamp  4 non-null      str  
dtypes: int64(1), str(5)
memory usage: 324.0 bytes


In [5]:
# STEP 1: Create output columns

df["reflection_theme"] = ""
df["quality_score"] = ""
df["actionability_score"] = ""
df["confidence_level"] = ""
df["needs_human_review"] = ""

df.head()


,participant_id,program_name,week_number,question,response,submission_timestamp,reflection_theme,quality_score,actionability_score,confidence_level,needs_human_review
0,P001,First-Time Manager Program,1,What leadership challenge did you face this week?,I struggled to delegate tasks because I was wo...,01-09-2025 10:15,,,,,
1,P002,First-Time Manager Program,1,What leadership challenge did you face this week?,Managing conflicting priorities was difficult ...,01-09-2025 11:02,,,,,
2,P003,First-Time Manager Program,1,What leadership challenge did you face this week?,I had trouble giving feedback to a team member...,01-09-2025 11:45,,,,,
3,P004,First-Time Manager Program,1,What leadership challenge did you face this week?,I was not clear about my role as a manager and...,01-09-2025 12:10,,,,,


In [6]:
# STEP 2: Theme detection logic

# Automatically assign a primary theme to each response using simple, explainable rules.

def detect_theme(text):
    text = text.lower()

    if "delegate" in text or "delegation" in text or "redo" in text:
        return "Delegation"
    elif "priority" in text or "priorities" in text or "overwhelmed" in text:
        return "Prioritization"
    elif "feedback" in text or "conversation" in text:
        return "Communication"
    elif "role" in text or "authority" in text:
        return "Role Clarity"
    else:
        return "Execution / Action Planning"
    
df["reflection_theme"] = df["response"].apply(detect_theme)

df[["response", "reflection_theme"]].head(10)


,response,reflection_theme
0,I struggled to delegate tasks because I was wo...,Delegation
1,Managing conflicting priorities was difficult ...,Prioritization
2,I had trouble giving feedback to a team member...,Communication
3,I was not clear about my role as a manager and...,Role Clarity


In [7]:
# STEP 3: Quality score logic

# How clear, specific, and understandable the response is.
# NOT whether it’s correct, successful, or has an action.

def calculate_quality(text):
    text = text.strip()

    # Very clear, specific explanation
    if len(text) >= 80:
        return "High"

    # Some clarity but not very detailed
    elif len(text) >= 40:
        return "Medium"

    # Very short / vague
    else:
        return "Low"


df["quality_score"] = df["response"].apply(calculate_quality)

df[["response", "quality_score"]].head(10)



,response,quality_score
0,I struggled to delegate tasks because I was wo...,High
1,Managing conflicting priorities was difficult ...,High
2,I had trouble giving feedback to a team member...,Medium
3,I was not clear about my role as a manager and...,High


In [8]:
# STEP 4: Actionability score logic
# Does this response enable forward movement, either by stating an action or showing an attempt to act?

def calculate_actionability(text):
        text = text.lower()

        # Clear forward action
        if "will" in text or "plan" in text or "next" in text:
            return "High"

        # Attempted action, but not fully resolved
        elif "tried" in text or "attempted" in text or "scheduled" in text:
            return "Medium"

        # No action, only description of problem
        else:
            return "Low"
        

df["actionability_score"] = df["response"].apply(calculate_actionability)

df[["response", "actionability_score"]].head(10)


,response,actionability_score
0,I struggled to delegate tasks because I was wo...,Low
1,Managing conflicting priorities was difficult ...,Low
2,I had trouble giving feedback to a team member...,Low
3,I was not clear about my role as a manager and...,Low


In [9]:
# STEP 5: Needs human review logic

def needs_human_review(row):
    if row["actionability_score"] == "Low":
        return "Yes"
    if row["quality_score"] == "Low":
        return "Yes"
    return "No"


df["needs_human_review"] = df.apply(needs_human_review, axis=1)

df[["response", "quality_score", "actionability_score", "needs_human_review"]].head(10)


,response,quality_score,actionability_score,needs_human_review
0,I struggled to delegate tasks because I was wo...,High,Low,Yes
1,Managing conflicting priorities was difficult ...,High,Low,Yes
2,I had trouble giving feedback to a team member...,Medium,Low,Yes
3,I was not clear about my role as a manager and...,High,Low,Yes


In [11]:
# STEP 6: Confidence level logic

def calculate_confidence(row):
    # If human review is required, confidence is lower
    if row["needs_human_review"] == "Yes":
        return "Medium"
    
    # Clear quality + some actionability → higher confidence
    if row["quality_score"] in ["High", "Medium"] and row["actionability_score"] in ["Medium", "High"]:
        return "High"
    
    return "Medium"


df["confidence_level"] = df.apply(calculate_confidence, axis=1)

df[["response", "quality_score", "actionability_score", "needs_human_review", "confidence_level"]].head(10)



,response,quality_score,actionability_score,needs_human_review,confidence_level
0,I struggled to delegate tasks because I was wo...,High,Low,Yes,Medium
1,Managing conflicting priorities was difficult ...,High,Low,Yes,Medium
2,I had trouble giving feedback to a team member...,Medium,Low,Yes,Medium
3,I was not clear about my role as a manager and...,High,Low,Yes,Medium


In [12]:
# STEP 7: Export final insights

df.to_csv(
    "sample_output.csv",
    index=False
)

print("Export completed: sample_output.csv")


Export completed: sample_output.csv


In [ ]:
import matplotlib.pyplot as plt

# Count reflections by theme
theme_counts = df["reflection_theme"].value_counts()

# Create bar chart
plt.figure(figsize=(8, 5))
theme_counts.plot(kind="bar")

# Titles and labels (keep simple)
plt.title("Leadership Reflection Themes – Current Snapshot")
plt.xlabel("Reflection Theme")
plt.ylabel("Number of Reflections")

# Rotate x-axis labels for readability
plt.xticks(rotation=30, ha="right")

# Clean layout
plt.tight_layout()

plt.show()
